# 意思決定分析と予測の活用｜第2部 決定分析の基礎
https://logics-of-blue.com/decision-analysis-and-forecast-book-support/

## 第6章 情報の価値

### Pythonによる分析の準備

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd

# DataFrameの全角文字の出力をきれいにする
pd.set_option('display.unicode.east_asian_width', True)
# 本文の数値とあわせるために、小数点以下3桁で丸める
pd.set_option('display.precision', 3)

In [2]:
# 最大値をとるインデックスを取得する。最大値が複数ある場合はすべて出力する。
def argmax_list(series):
    return(list(series[series == series.max()].index))

In [3]:
# 期待金額最大化に基づく意思決定を行う関数
def max_emv(probs, payoff_table):
    emv = payoff_table.mul(probs, axis=0).sum()
    max_emv = emv.max()
    a_star = argmax_list(emv)
    return(pd.Series([a_star, max_emv], index=['選択肢', '期待金額']))

### 同時分布と利得行列

In [4]:
# 利得行列
payoff = pd.DataFrame({
    '1台': [300, 300],
    '2台': [700, -300]
})
payoff.index = ['好況', '不況']
print(payoff)

      1台  2台
好況  300  700
不況  300 -300


In [5]:
# 同時分布
joint_forecast_state = pd.DataFrame({
    '好況予測': [0.35, 0.1],
    '不況予測': [0.05, 0.5]
})
joint_forecast_state.index = ['好況', '不況']
print(joint_forecast_state)

      好況予測  不況予測
好況      0.35      0.05
不況      0.10      0.50


### 条件付き期待値に基づく意思決定

#### 周辺分布を求める

In [6]:
# 予測の周辺分布
marginal_forecast = joint_forecast_state.sum(axis=0)
marginal_forecast

好況予測    0.45
不況予測    0.55
dtype: float64

In [7]:
# 自然の状態の周辺分布
marginal_state = joint_forecast_state.sum(axis=1)
marginal_state

好況    0.4
不況    0.6
dtype: float64

#### 条件付分布を求める

In [8]:
# 予測が得られた後の条件付き分布
conditional_forecast = joint_forecast_state.div(marginal_forecast, axis=1)
print(conditional_forecast)

      好況予測  不況予測
好況     0.778     0.091
不況     0.222     0.909


#### 条件付き期待値に基づく意思決定

In [9]:
info_decision = \
    conditional_forecast.apply(max_emv, axis=0, payoff_table=payoff)
print(info_decision)

         好況予測 不況予測
選択肢      [2台]    [1台]
期待金額      478      300


### 情報を使わない意思決定との比較

In [10]:
naive_decision = max_emv(marginal_state, payoff)
naive_decision

選択肢      [1台]
期待金額      300
dtype: object

In [11]:
emv_naive = naive_decision['期待金額']
print(f'予測を使わないときの期待金額: {emv_naive:.3g}万円')

予測を使わないときの期待金額: 300万円


### 情報の価値：事後分析

In [12]:
post_value_boom = info_decision.loc['期待金額', '好況予測'] - emv_naive
print(f'好況予測が出たときの期待金額の差: {post_value_boom:.3g}万円')

好況予測が出たときの期待金額の差: 178万円


In [13]:
post_value_slump = info_decision.loc['期待金額', '不況予測'] - emv_naive
print(f'不況予測が出たときの期待金額の差: {post_value_slump:.3g}万円')

不況予測が出たときの期待金額の差: -5.68e-14万円


### 情報の価値：事前分析

In [14]:
emv_forecast = info_decision.loc['期待金額'].mul(marginal_forecast).sum()
print(f'情報を使ったときの期待金額: {emv_forecast:.3g}万円')

情報を使ったときの期待金額: 380万円


In [15]:
# 同時分布を使って計算しても良い
joint_forecast_state.apply(
    max_emv, axis=0, payoff_table=payoff).loc['期待金額'].sum()

380.0

In [16]:
ante_value = emv_forecast - emv_naive
print(f'情報の価値: {ante_value:.3g}万円')

情報の価値: 80万円


### 完全情報と完全情報の価値

In [17]:
# 自然の状態別の最大利得
payoff.max(axis=1)

好況    700
不況    300
dtype: int64

In [18]:
# 自然の状態にあわせて利得を最大にする行動をとったときの期待金額
emv_perfect = payoff.max(axis=1).mul(marginal_state).sum()
emv_perfect

460.0

In [19]:
perfect_information_value = emv_perfect - emv_naive
print(f'完全情報の価値: {perfect_information_value:.3g}万円')

完全情報の価値: 160万円


### 情報の価値の比較

In [20]:
# 「コインを投げて表なら好況、裏なら不況と主張する」情報
coin_result = pd.DataFrame({
    '好況予測': [0.2, 0.3],
    '不況予測': [0.2, 0.3]
})
coin_result.index = ['好況', '不況']
print(coin_result)

      好況予測  不況予測
好況       0.2       0.2
不況       0.3       0.3


In [21]:
# コイン投げの結果を使ったときの期待金額
emv_coin = coin_result.apply(
    max_emv, axis=0, payoff_table=payoff).loc['期待金額'].sum()
emv_coin

300.0

In [22]:
# 予測値と実測値が常に逆になる情報
perversity_forecast = pd.DataFrame({
    '好況予測': [0, 0.6],
    '不況予測': [0.4, 0]
})
perversity_forecast.index = ['好況', '不況']
print(perversity_forecast)

      好況予測  不況予測
好況       0.0       0.4
不況       0.6       0.0


In [23]:
# 予測値と実測値が常に逆になる情報を使ったときの期待金額
emv_perversity = perversity_forecast.apply(
    max_emv, axis=0, payoff_table=payoff).loc['期待金額'].sum()
emv_perversity

460.0

In [24]:
efficiency = ante_value / perfect_information_value
print(f'情報の有効性: {efficiency:.3g}')

情報の有効性: 0.5
